# 05 Success vs failure transfer tasks

Section covered: 7 (toy intuition to realistic split).

**Hypothesis**: related source-target pairs transfer better than unrelated pairs.

In [ ]:
from pathlib import Path
from copy import deepcopy
import subprocess
import yaml
import pandas as pd
import matplotlib.pyplot as plt

try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    ROOT = Path.cwd().resolve()
LOGS = ROOT / 'outputs' / 'logs'
FIGS = ROOT / 'outputs' / 'figures'
LOGS.mkdir(parents=True, exist_ok=True)
FIGS.mkdir(parents=True, exist_ok=True)

def run_config(config_path: Path, use_progress: bool = True):
    cmd = ['python', str(ROOT / 'scripts' / 'run_transfer.py'), '--config', str(config_path)]
    if use_progress:
        cmd.append('--use-progress')
    subprocess.run(cmd, cwd=ROOT, check=True)

def read_method(name: str) -> pd.DataFrame:
    return pd.read_csv(LOGS / f'transfer_{name}.csv')

def deep_update(base: dict, patch: dict) -> dict:
    for k, v in patch.items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            deep_update(base[k], v)
        else:
            base[k] = v
    return base

def apply_fast_dev_profile(cfg: dict) -> dict:
    cfg = deepcopy(cfg)
    data = cfg.setdefault('data', {})
    train = cfg.setdefault('train', {})

    if 'source_train_per_class' in data:
        data['source_train_per_class'] = min(int(data['source_train_per_class']), 220)
    if 'source_test_per_class' in data:
        data['source_test_per_class'] = min(int(data['source_test_per_class']), 80)
    if 'target_train_per_class' in data:
        data['target_train_per_class'] = min(int(data['target_train_per_class']), 30)
    if 'target_test_per_class' in data:
        data['target_test_per_class'] = min(int(data['target_test_per_class']), 80)
    if 'probe_per_class' in data:
        data['probe_per_class'] = min(int(data['probe_per_class']), 40)

    data['batch_size'] = min(int(data.get('batch_size', 128)), 64)
    data['num_workers'] = 0

    if 'source_epochs' in train:
        train['source_epochs'] = min(int(train['source_epochs']), 2)
    if 'target_epochs' in train:
        train['target_epochs'] = min(int(train['target_epochs']), 3)

    train['gradual_schedule'] = {
        '1': ['backbone.layer4'],
        '2': ['backbone.layer3'],
    }
    return cfg

def make_profiled_config(base_name: str, notebook_tag: str, fast_dev_run: bool, overrides: dict | None = None) -> Path:
    cfg = yaml.safe_load((ROOT / 'configs' / base_name).read_text())
    if fast_dev_run:
        cfg = apply_fast_dev_profile(cfg)
    if overrides:
        cfg = deep_update(cfg, deepcopy(overrides))

    suffix = 'fast' if fast_dev_run else 'full'
    out = ROOT / 'configs' / f'tmp_{notebook_tag}_{suffix}.yaml'
    out.write_text(yaml.safe_dump(cfg, sort_keys=False))
    return out


In [ ]:
FAST_DEV_RUN = False
related_cfg = make_profiled_config(
    base_name='transfer_core_related.yaml',
    notebook_tag='05_related',
    fast_dev_run=FAST_DEV_RUN,
    overrides={'methods': ['gradual_unfreeze']},
)
run_config(related_cfg, use_progress=not FAST_DEV_RUN)
related = read_method('gradual_unfreeze').copy()
related['scenario'] = 'related'

unrelated_cfg = make_profiled_config(
    base_name='transfer_unrelated_failure.yaml',
    notebook_tag='05_unrelated',
    fast_dev_run=FAST_DEV_RUN,
    overrides={'methods': ['gradual_unfreeze']},
)
run_config(unrelated_cfg, use_progress=not FAST_DEV_RUN)
unrelated = read_method('gradual_unfreeze').copy()
unrelated['scenario'] = 'unrelated'

both = pd.concat([related, unrelated], ignore_index=True)
both.head()


In [ ]:
plt.figure(figsize=(6.5, 3.5))
for scenario, df in both.groupby('scenario'):
    plt.plot(df['epoch'], df['target_test_acc'], marker='o', label=scenario)
plt.title('Related vs unrelated transfer (same adaptation policy)')
plt.xlabel('epoch')
plt.ylabel('target_test_acc')
plt.grid(alpha=0.2)
plt.legend(frameon=False)


In [ ]:
summary = both.groupby('scenario').agg(
    final_target_acc=('target_test_acc', 'last'),
    final_retention=('source_retention_acc', 'last'),
    final_drift=('feature_drift', 'last'),
).reset_index()
summary


## Interpretation

Task alignment is visible as a shift in the full stability profile, not only endpoint accuracy.